**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed May 24 11:56:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
yolov3


**1) Clone the Darknet**



In [ ]:
%cd /mydrive/yolov3/
!git clone https://github.com/AlexeyAB/darknet

/content/gdrive/My Drive/yolov3
Cloning into 'darknet'...
remote: Enumerating objects: 15521, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15521 (delta 0), reused 4 (delta 0), pack-reused 15514
Receiving objects: 100% (15521/15521), 14.17 MiB | 12.29 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.
Updating files: 100% (2058/2058), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/gdrive/MyDrive/yolov3/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3/training"

In [ ]:
!echo "Scalpel" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-05-24 12:03:12--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  61.9MB/s    in 2.5s    

2023-05-24 12:03:15 (61.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/obj.zip -d data/obj

Archive:  /mydrive/yolov3/obj.zip
   creating: data/obj/obj/
  inflating: data/obj/obj/bisturi1.jpg  
  inflating: data/obj/obj/bisturi1.txt  
  inflating: data/obj/obj/bisturi10.jpg  
  inflating: data/obj/obj/bisturi10.txt  
  inflating: data/obj/obj/bisturi100.jpg  
  inflating: data/obj/obj/bisturi100.txt  
  inflating: data/obj/obj/bisturi101.jpg  
  inflating: data/obj/obj/bisturi101.txt  
  inflating: data/obj/obj/bisturi102.jpg  
  inflating: data/obj/obj/bisturi102.txt  
  inflating: data/obj/obj/bisturi103.jpg  
  inflating: data/obj/obj/bisturi103.txt  
  inflating: data/obj/obj/bisturi104.jpg  
  inflating: data/obj/obj/bisturi104.txt  
  inflating: data/obj/obj/bisturi105.jpg  
  inflating: data/obj/obj/bisturi105.txt  
  inflating: data/obj/obj/bisturi106.jpg  
  inflating: data/obj/obj/bisturi106.txt  
  inflating: data/obj/obj/bisturi107.jpg  
  inflating: data/obj/obj/bisturi107.txt  
  inflating: data/obj/obj/bisturi108.jpg  
  inflating: data/obj/obj/bisturi108.txt  

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/obj/*.jpg")
print(images_list)

['data/obj/obj/bisturi1.jpg', 'data/obj/obj/bisturi10.jpg', 'data/obj/obj/bisturi100.jpg', 'data/obj/obj/bisturi101.jpg', 'data/obj/obj/bisturi102.jpg', 'data/obj/obj/bisturi103.jpg', 'data/obj/obj/bisturi104.jpg', 'data/obj/obj/bisturi105.jpg', 'data/obj/obj/bisturi106.jpg', 'data/obj/obj/bisturi107.jpg', 'data/obj/obj/bisturi108.jpg', 'data/obj/obj/bisturi109.jpg', 'data/obj/obj/bisturi11.jpg', 'data/obj/obj/bisturi110.jpg', 'data/obj/obj/bisturi111.jpg', 'data/obj/obj/bisturi112.jpg', 'data/obj/obj/bisturi113.jpg', 'data/obj/obj/bisturi114.jpg', 'data/obj/obj/bisturi115.jpg', 'data/obj/obj/bisturi116.jpg', 'data/obj/obj/bisturi117.jpg', 'data/obj/obj/bisturi118.jpg', 'data/obj/obj/bisturi119.jpg', 'data/obj/obj/bisturi12.jpg', 'data/obj/obj/bisturi120.jpg', 'data/obj/obj/bisturi121.jpg', 'data/obj/obj/bisturi122.jpg', 'data/obj/obj/bisturi123.jpg', 'data/obj/obj/bisturi124.jpg', 'data/obj/obj/bisturi125.jpg', 'data/obj/obj/bisturi126.jpg', 'data/obj/obj/bisturi127.jpg', 'data/obj/ob

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 243: 1.423914, 1.704984 avg loss, 0.000003 rate, 5.023107 seconds, 15552 images, 5.739849 hours left
Loaded: 0.000095 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.436296), count: 3, class_loss = 1.141315, iou_loss = 1.376993, total_loss = 2.518307 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.450358), count: 1, class_loss = 0.585907, iou_loss = 0.403073, total_loss = 0.988979 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.229028, iou_loss = 0.000000, total_loss = 0.229028 
 total_bbox = 15282, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.525072), count: 2, class_loss = 0.901993, iou_loss = 0.665298, total_loss = 1.567291 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.487380), count: 2

In [ ]:
#set your custom cfg to test mode 
%cd darknet
!sed -i 's/batch=64/batch=1/' yolov3_training.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom.cfg
%cd ..

[Errno 2] No such file or directory: 'darknet'
/
sed: can't read yolov3_training.cfg: No such file or directory
sed: can't read yolov4-custom.cfg: No such file or directory
/
